In [92]:
import numpy as np
import torch
from __future__ import print_function
from torch import nn
from torch import autograd
import torch.optim as optim
import torch.nn.functional as F
import math
from torch.autograd import Variable
import copy

In [3]:
""""#import pyedflib
import numpy as np
f = pyedflib.EdfReader("edf1")
n = f.signals_in_file
signal_labels = f.getSignalLabels()
sigbufs = np.zeros((n, f.getNSamples()[0]))
for i in np.arange(n):
    sigbufs[i, :] = f.readSignal(i)""""

ImportError: No module named pyedflib

In [ ]:
sample = sigbufs[:, 201:1401]

In [152]:
class LikeDislikeModel(nn.Module):
    def __init__(self, 
                 batch_size,
                 in_channels,
                 out_size,
                 L_sizes = [300, 200, 300],
                 fc_size = 20,
                 n_local_pred = 3,
                 f_sizes = [3, 3, 5], 
                 channels = [24, 16, 8], 
                 strides = [2, 2, 3], 
                 use_cuda = True):
        
        super(LikeDislikeModel, self).__init__()
        
        self.in_channels = in_channels
        self.batch_size = batch_size
        self.fc_size = fc_size
        self.out_size = out_size
        self.n_local_pred = n_local_pred
        self.use_cuda = use_cuda
        self.L_sizes = L_sizes
        
        self.f_sizes = f_sizes
        self.channels = channels
        self.strides = strides
        self.len_sizes = self._len_sizes()
        
        self.conv1 = nn.ModuleList()
        self.conv2 = nn.ModuleList()
        self.conv3 = nn.ModuleList()
        self.fc1 = nn.ModuleList()
        self.fc2 = nn.ModuleList()
        
        # TODO : add batchnormalization for each conv 
        #self.batchnorm = nn.ModuleList()
        
        
        
        
        for i in range (n_local_pred):
            
            self.conv1.append(nn.Conv1d(in_channels, channels[0], f_sizes[0], strides[0]))
            self.conv2.append(nn.Conv1d(channels[0], channels[1], f_sizes[1], strides[1]))               
            self.conv3.append(nn.Conv1d(channels[1], channels[2], f_sizes[2], strides[2]))
            
            self.fc1.append(nn.Linear(self.len_sizes[i][-1] * self.channels[-1], fc_size))
            self.fc2.append(nn.Linear(fc_size, out_size))
            
            
                              
        self.dropout = nn.Dropout(p=0.5)
        
    def _len_sizes(self):
        len_sizes = []
        for i in range (self.n_local_pred):
            temp = []
            for j in range (len(self.L_sizes)):
                if j == 0:
                    temp.append(int(math.floor((self.L_sizes[i] - self.f_sizes[j])/self.strides[j] + 1)))
                else:
                    temp.append(int(math.floor((temp[j-1] - self.f_sizes[j])/self.strides[j] + 1)))
            len_sizes.append(copy.deepcopy(temp))
        return len_sizes

    def preprocessing(self, input_batch):
        nns = []
        nns.append(input_batch[:, :, 200:500])
        nns.append(input_batch[:, :, 500:700])
        nns.append(input_batch[:, :, 700:1000])
        for i in range (self.n_local_pred):
            zero_mean = nns[i] - nns[i].mean()
            for n in range(input_batch.shape[0]):
                nns[i][n] = zero_mean[n, :, :]/(np.std(zero_mean[n, :, :],axis = 0) + 0.1)
                
            if self.use_cuda:
                nns[i] = Variable(torch.FloatTensor(copy.deepcopy(nns[i]))).cuda()
            else:
                nns[i] = Variable(torch.FloatTensor(copy.deepcopy(nns[i])))
                

        return nns

    def createLocalNetworks(self, nns):
        predict = []
        for nn_num in range( self.n_local_pred):
            conv1 = self.conv1[nn_num](nns[nn_num])
            relu1 = F.relu(conv1)
            print (conv1.size())

            conv2 = self.conv2[nn_num](relu1)
            relu2 = F.relu(conv2)
            print (conv2.size())
            
            conv3 = self.conv3[nn_num](relu2)
            relu3 = F.relu(conv3)
            print (conv2.size())
            
            relu3 = relu3.view(self.batch_size, -1)
            
            fc1 = self.fc1[nn_num](relu3)
            relu4 = F.relu(fc1)

            do = self.dropout(relu4)

            fc2 = self.fc2[nn_num](do)
            
            predict.append(F.log_softmax(fc2))
        return predict


    def forward(self, input_batch):
        print (123)
        nns  = self.preprocessing(input_batch)
        predict = self.createLocalNetworks(nns)
        return predict

In [153]:
model = LikeDislikeModel(5, 65, 3, use_cuda = False)
if model.use_cuda:
    model.cuda()
output = model(sample)

123
torch.Size([5, 24, 149])
torch.Size([5, 16, 74])
torch.Size([5, 16, 74])
torch.Size([5, 24, 99])
torch.Size([5, 16, 49])
torch.Size([5, 16, 49])
torch.Size([5, 24, 149])
torch.Size([5, 16, 74])
torch.Size([5, 16, 74])


/home/rauf/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:108: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [154]:
output

[Variable containing:
 -1.1709 -1.1319 -1.0011
 -1.1730 -1.1416 -0.9908
 -1.1717 -1.1321 -1.0002
 -1.1725 -1.1409 -0.9919
 -1.1769 -1.1461 -0.9838
 [torch.FloatTensor of size 5x3], Variable containing:
 -1.1959 -0.9043 -1.2285
 -1.1745 -0.9016 -1.2549
 -1.1669 -0.9157 -1.2433
 -1.1571 -0.9256 -1.2403
 -1.1541 -0.9103 -1.2650
 [torch.FloatTensor of size 5x3], Variable containing:
 -1.2286 -1.1004 -0.9819
 -1.2012 -1.1221 -0.9846
 -1.2127 -1.1524 -0.9500
 -1.1965 -1.1335 -0.9786
 -1.1968 -1.1083 -1.0004
 [torch.FloatTensor of size 5x3]]